# Resolved a sudoku thanks to CSP

Imports

In [9]:
import time
import copy
import os
from collections import deque

## Let's create the game and rules

Create the grill

In [10]:
easy_sudoku_grid = [
    [0, 0, 0, 2, 6, 0, 7, 0, 1],
    [6, 8, 0, 0, 7, 0, 0, 9, 0],
    [1, 9, 0, 0, 0, 4, 5, 0, 0],
    [8, 2, 0, 1, 0, 0, 0, 4, 0],
    [0, 0, 4, 6, 0, 2, 9, 0, 0],
    [0, 5, 0, 0, 0, 3, 0, 2, 8],
    [0, 0, 9, 3, 0, 0, 0, 7, 4],
    [0, 4, 0, 0, 5, 0, 0, 3, 0],
    [7, 0, 3, 0, 1, 8, 0, 0, 0]
]
medium_sudoku_grid = [
    [0, 0, 0, 0, 6, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 9, 0, 0, 0],
    [0, 0, 8, 0, 0, 0, 0, 6, 0],
    [0, 0, 0, 0, 0, 0, 4, 0, 0],
    [0, 0, 0, 8, 0, 3, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 9, 0, 0, 0, 0, 3, 0, 0],
    [0, 0, 0, 4, 0, 7, 0, 0, 0],
    [0, 0, 0, 0, 2, 0, 0, 0, 0]
]

hard_sudoku_grid = [
    [8, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 3, 6, 0, 0, 0, 0, 0],
    [0, 7, 0, 0, 9, 0, 2, 0, 0],
    [0, 5, 0, 0, 0, 7, 0, 0, 0],
    [0, 0, 0, 0, 4, 5, 7, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 3, 0],
    [0, 0, 1, 0, 0, 0, 0, 6, 8],
    [0, 0, 8, 5, 0, 0, 0, 1, 0],
    [0, 9, 0, 0, 0, 0, 4, 0, 0]
]

def initiate_sudoku_grid(name):
    if name == 'easy':
        return easy_sudoku_grid
    elif name == 'medium':
        return medium_sudoku_grid
    elif name == 'hard':
        return hard_sudoku_grid
    else:
        print('Invalid name of the sudoku grid. Please choose between "easy" and "hard"')
        return None


#function in order to plot the grid
def print_grid(grid): 
    print()
    for row in grid:
        print(" ".join(str(x) if x != 0 else 'x' for x in row))
    print()
print_grid(easy_sudoku_grid)

def save_grid(grid, counter,already_saved):
    
    grid_string = already_saved  +'\n' f'Grid {counter}:\n'
    for row in grid:
        grid_string += " ".join(str(x) if x != 0 else 'x' for x in row) + '\n'
    grid_string += '\n'
    return grid_string

def write_to_file(grid_string, method):
    file_path = f"sudoku_grid_with_{method}.txt"
    #delete the file if it already exists
    if os.path.exists(file_path):
        os.remove(file_path)
    with open(file_path, 'a') as f:
        f.write(grid_string)


x x x 2 6 x 7 x 1
6 8 x x 7 x x 9 x
1 9 x x x 4 5 x x
8 2 x 1 x x x 4 x
x x 4 6 x 2 9 x x
x 5 x x x 3 x 2 8
x x 9 3 x x x 7 4
x 4 x x 5 x x 3 x
7 x 3 x 1 8 x x x



Now check for each case if the number can be place

In [11]:
def is_valid(grid,row,col,number):
    #check the line
    for i in range(0,9):
        if grid[row][i] == number:
            return False
    #check the column
    for i in range(0,9):
        if grid[i][col] == number:
            return False
    #check the square
    #get in which square the number is stored 
    #for the row we will get the first row of the square
    square_row = row - row % 3
    #for the column we will get the first column of the square
    square_col = col - col % 3
    #now we can check
    for i in range(square_row,square_row+3): 
        for j in range(square_col,square_col+3):#3x3 square
            if grid[i][j] == number:
                return False
            
    return True

#tests
#for row check: 2 in (0,0) should return False
print(is_valid(easy_sudoku_grid,0,0,2))
#for column check: 7 in (0,0) should return False
print(is_valid(easy_sudoku_grid,0,0,7))
#for square check: 9 in (0,0) should return False
print(is_valid(easy_sudoku_grid,0,0,9))
#For global check 3 in (0,0) should return True
print(is_valid(easy_sudoku_grid,0,0,3))


False
False
False
True


now check if the grid is completed

In [12]:
def is_completed(grid):
    for row in grid:
        if 0 in row:
            return False
    return True

## Let's start CSP

Backtracking solution baseline

In [13]:
def simple_backtracking(grid,counter,grid_string):
    #loop in each cells
    for i in range(0,9):
        for j in range(0,9):
            #check if the cells is empty or not
            if grid[i][j] == 0:
                #assign a number that is valid
                for number in range(1,10):
                    if is_valid(grid,i,j,number):
                        grid_string = save_grid(grid,counter,grid_string)
                        #if valid edit the grid and call the function recursively
                        grid[i][j] = number
                        
                        counter, solved = simple_backtracking(grid,counter+1,grid_string)
                        
                        if solved:
                            grid_string = save_grid(grid,counter,grid_string)
                            write_to_file(grid_string, 'backtracking_with_forward_checking')
                            return counter,True
                        
                        #if we arrived here this means that their is no solution for the next cell to fill, so backtracking (and we switch the cell back to 0)
                        #backtracking
                        grid[i][j] = 0
                return counter,False
            
    if is_completed(grid):
        print_grid(grid)
        grid_string = save_grid(grid,counter,grid_string)
        write_to_file(grid_string, 'simple_backtracking')
        # print(f"Solved with : {counter} iterations")
        return counter,True
    return counter,False

##  Reduction

First method for problem reduction :  Forward checking

In [14]:
def forward_checking(grid,row,col,domains):
    number = grid[row][col]
    if number != 0:
        #update the domains of the cells in the same row
        
        for i in range(9):
            if grid[row][i] == 0 and number in domains[row][i]:
                domains[row][i].remove(number)
                
        #update the domains of the cells in the same column
        for j in range (9):
            if grid[j][col] == 0 and number in domains[j][col]:
                domains[j][col].remove(number)
        
        #update the domains of the cells in the same square
        
        #for the row we will get the first row of the square
        square_row = row - row % 3
        #for the column we will get the first column of the square
        square_col = col - col % 3
        for k in range(square_row,square_row+3):
            for l in range(square_col,square_col+3):
                if grid[k][l] == 0 and number in domains[k][l]:
                    domains[k][l].remove(number)
        
    return domains

def create_domains(grid):
    # Function which creates domains at the beginning with the starting grid
    # Default domains
    domains = [[[1,2,3,4,5,6,7,8,9] for _ in range(9)] for _ in range(9)]
    
    # Loop in each cell
    for i in range(0,9):
        for j in range(0,9):
            if grid[i][j] != 0:
                number = grid[i][j]
                # Update the domains of the cells in the same row
                for k in range(9):
                    if number in domains[i][k]:
                        domains[i][k].remove(number)
                # Update the domains of the cells in the same column
                for l in range(9):
                    if number in domains[l][j]:
                        domains[l][j].remove(number)
                # Update the domains of the cells in the same square
                square_row = i - i % 3
                square_col = j - j % 3
                for m in range(square_row, square_row + 3):
                    for n in range(square_col, square_col + 3):
                        if number in domains[m][n]:
                            domains[m][n].remove(number)
    return domains



#This function allow to update back the domains when we backtrack
def update_domains(row,col,number,domains):
    #update the domains of the cells in the same row
    for i in range(9):
        if number  not in domains[row][i]:
            domains[row][i].append(number)
    #update the domains of the cells in the same column
    for j in range (9):
        if number not in domains[j][col]:
            domains[j][col].append(number)
    #update the domains of the cells in the same square
    square_row = row - row % 3
    square_col = col - col % 3
    for k in range(square_row,square_row+3):
        for l in range(square_col,square_col+3):
            if number not in domains[k][l]:
                domains[k][l].append(number)
    return domains


def find_next_empty(grid):
    for i in range(9):
        for j in range(9):
            if grid[i][j] == 0:
                return i,j
    return None,None

#DWORK BUT MORE ITERATION ? WHY
def backtracking_with_forward_checking(grid,domains,counter,grid_string):


    
    row,col = find_next_empty(grid)
    # print("============================================")
    # print(f"counter : {counter}")
    # print(f"row: {row}")
    # print_grid(grid)
    # print("============================================")
    if row is None:
        print("in row is None")
        print_grid(grid)
        grid_string = save_grid(grid,counter,grid_string)
        write_to_file(grid_string, 'backtracking_with_forward_checking')
        return counter,True
    
    for number in domains[row][col]:
        if is_valid (grid,row,col,number):
            grid_string = save_grid(grid,counter,grid_string)
            grid[row][col] = number
            new_domains = forward_checking(grid, row, col, [row[:] for row in domains])
            counter,solved = backtracking_with_forward_checking(grid,new_domains,counter+1,grid_string)
            if solved:
                grid_string = save_grid(grid,counter,grid_string)
                write_to_file(grid_string, 'backtracking_with_forward_checking')
                return counter,True
            grid[row][col] = 0
            domains = update_domains(row,col,number,domains)
    return counter,False


Second method for problem reduction :  AC-3

In [15]:
#a function in order to get the neighbors of a cell
def get_neighbors(row, col):
    #get all the cells from the columns,rows,square of the cell
    neighbors = set()
    for k in range(9):
        neighbors.add((row, k))
        neighbors.add((k, col))
    square_row, square_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(square_row, square_row + 3):
        for j in range(square_col, square_col + 3):
            neighbors.add((i, j))
    neighbors.discard((row, col))  # Remove itself
    return neighbors



#a function in order to remove the inconsistent values ()
def remove_inconsistent_values(domains, row, col):
    removed = False
    # If the domain has only one value, we remove it from the neighbors
    if len(domains[row][col]) == 1:
        value = domains[row][col][0]
        for (i, j) in get_neighbors(row, col):
            if value in domains[i][j]:
                domains[i][j].remove(value)
                removed = True
    return removed


#ac3 algorithm
def ac3(domains):
    #a queue of all the arcs in the CSP
    queue = deque([(xi, xj) for xi in range(9) for xj in range(9) if len(domains[xi][xj]) > 1])
    while queue:
        (xi, xj) = queue.popleft()
        if remove_inconsistent_values(domains, xi, xj):
            if not domains[xi][xj]:  # Empty domain means fail/error
                return False
            # Add concerned neighbors to the queue
            for neighbor in get_neighbors(xi, xj):
                queue.append(neighbor)
    return True

def backtracking_with_ac3(grid, domains, counter, grid_string):
    # Step 1: Initialize domains if they haven't been passed in yet
    if domains is None:
        domains = create_domains(grid)
        if not ac3(domains):  # AC-3 preprocessing; fails if any domain becomes empty
            return counter, False

    # Step 2: Find the next empty cell for backtracking
    row, col = find_next_empty(grid)
    if row is None:  # Puzzle is solved if no empty cell remains
        print("Solution found:")
        print_grid(grid)
        grid_string = save_grid(grid, counter, grid_string)
        write_to_file(grid_string, 'backtracking_with_ac3')
        return counter, True

    # Step 3: Perform backtracking using AC-3 reduced domains
    for number in domains[row][col]:  # Iterate over possible values in the domain
        if is_valid(grid, row, col, number):
            grid[row][col] = number  # Place the number
            grid_string = save_grid(grid, counter, grid_string)

            # Make a deep copy of domains to pass to the next recursion level
            new_domains = [row[:] for row in domains]
            new_domains = update_domains(row, col, number, new_domains)  # Update domains after placing number

            # Recursive call with updated counter
            counter, solved = backtracking_with_ac3(grid, new_domains, counter + 1, grid_string)
            if solved:
                grid_string = save_grid(grid, counter, grid_string)
                write_to_file(grid_string, 'backtracking_with_ac3')
                return counter, True

            # Backtrack: remove the number and reset the cell
            grid[row][col] = 0

    return counter, False

## Basic search strategies and/or Stochastic search methods (need to rename properly this one)

## Function(s) in order to compare the efficacity of each method

time and iterations

In [ ]:
def measure_time_and_iterations(func,*args):
    start_time = time.time()
    iterations = func(*args)[0]
    end_time = time.time()
    executive_time = end_time - start_time
    return executive_time, iterations

def testing(grid_difficulty):
    methods= ['backtracking_with_forward_checking','simple_backtracking','ac3_backtracking']
    for  method in methods:
        
        original_grid = initiate_sudoku_grid(grid_difficulty)
        if method == 'simple_backtracking':
            grid = copy.deepcopy(original_grid)
            grid_string = ""
            simple_version_time = measure_time_and_iterations(simple_backtracking,grid,0,grid_string)
            print(f"The simple version take {simple_version_time[0]} seconds and {simple_version_time[1]} iterations to solve the sudoku")
        elif method == 'backtracking_with_forward_checking':
            grid = copy.deepcopy(original_grid)
            grid_string = ""
            domains_for_forward_checking = create_domains(grid)
            forward_checking_time = measure_time_and_iterations(backtracking_with_forward_checking,grid,domains_for_forward_checking,0,grid_string)
            print(f"The forward checking version take {forward_checking_time[0]} seconds and {forward_checking_time[1]} iterations to solve the sudoku")
        elif method == 'ac3_backtracking':
            grid = copy.deepcopy(original_grid)
            grid_string = ""
            domains_for_ac3 = create_domains(grid)
            ac3_time =  measure_time_and_iterations(backtracking_with_ac3, grid, domains_for_ac3, 0, grid_string)
            print(f"The ac3 version take {ac3_time[0]} seconds and {ac3_time[1]} iterations to solve the sudoku")
            

testing('hard')

#forward checking less efficient that normal ? only for hard not for easy and medium (put it in the report)
#ac3 dont seem to work (same result as backtracking classic everytime)
#save_grid dont have all the grids...

in row is None

8 1 2 7 5 3 6 4 9
9 4 3 6 8 2 1 7 5
6 7 5 4 9 1 2 8 3
1 5 4 2 3 7 8 9 6
3 6 9 8 4 5 7 2 1
2 8 7 1 6 9 5 3 4
5 2 1 9 7 4 3 6 8
4 3 8 5 2 6 9 1 7
7 9 6 3 1 8 4 5 2

The forward checking version take 2.8458449840545654 seconds and 76745 iterations to solve the sudoku

8 1 2 7 5 3 6 4 9
9 4 3 6 8 2 1 7 5
6 7 5 4 9 1 2 8 3
1 5 4 2 3 7 8 9 6
3 6 9 8 4 5 7 2 1
2 8 7 1 6 9 5 3 4
5 2 1 9 7 4 3 6 8
4 3 8 5 2 6 9 1 7
7 9 6 3 1 8 4 5 2

The simple version take 1.4426565170288086 seconds and 49558 iterations to solve the sudoku
Solution found:

8 1 2 7 5 3 6 4 9
9 4 3 6 8 2 1 7 5
6 7 5 4 9 1 2 8 3
1 5 4 2 3 7 8 9 6
3 6 9 8 4 5 7 2 1
2 8 7 1 6 9 5 3 4
5 2 1 9 7 4 3 6 8
4 3 8 5 2 6 9 1 7
7 9 6 3 1 8 4 5 2

The ac3 version take 1.7248878479003906 seconds and 49558 iterations to solve the sudoku
